# 3-2 Dataset と DataLoader の実装

## フォルダ準備
make_folders_and_data_downloads.ipynb を実行してフォルダの準備をする．  
また，これとは別に[このページ](https://drive.google.com/uc?id=12eN6SpnawYuQmD1k9VgVW3QSgPR6hICc&export=download)から pspnet50_ADE20K.pth をダウンロードし，weights フォルダに格納する．

## 画像データとアノテーションデータへのファイルパスリストを作成
はじめに，画像データとアノテーションデータへのファイルパスを格納したリストを作成する．
基本的に前章と同じように作成する．

In [1]:
import os
from PIL import Image
import torch.utils.data as data

In [2]:
def make_datapath_list(rootpath):
    """
    学習、検証の画像データとアノテーションデータへのファイルパスリストを作成する。

    Parameters
    ----------
    rootpath : str
        データフォルダへのパス
    
    Returns
    -------
    ret : train_img_list, train_anno_list, val_img_list, val_anno_list
        データへのパスを格納したリスト
    """
    
    # 画像ファイルとアノテーションファイルへのパスのテンプレートを作成
    img_path_template = os.path.join(rootpath, "JPEGImages", "%s.jpg")
    anno_path_template = os.path.join(rootpath, "SegmentationClass", "%s.png")
    
    # 訓練と検証それぞれのファイル名を取得
    train_id_names = os.path.join(rootpath, "ImageSets/Segmentation/train.txt")
    val_id_names = os.path.join(rootpath, "ImageSets/Segmentation/val.txt")
    
    # 訓練データの画像ファイルとアノテーションファイルへのパスリストを作成
    train_img_list = list()
    train_anno_list = list()
    for line in open(train_id_names):
        file_id = line.strip()                     # 空白スペースと改行を除去
        img_path = (img_path_template % file_id)   #画像のパス
        anno_path = (anno_path_template % file_id) # アノテーションのパス
        train_img_list.append(img_path)
        train_anno_list.append(anno_path)
        
    # 検証データの画像ファイルとアノテーションファイルへのパスリストを作成
    val_img_list = list()
    val_anno_list = list()
    for line in open(val_id_names):
        file_id = line.strip()                     # 空白スペースと改行を除去
        img_path = (img_path_template % file_id)   #画像のパス
        anno_path = (anno_path_template % file_id) # アノテーションのパス
        val_img_list.append(img_path)
        val_anno_list.append(anno_path)
        
    return train_img_list, train_anno_list, val_img_list, val_anno_list

In [3]:
# 動作確認
rootpath = "./data/VOCdevkit/VOC2012/"
train_img_list, train_anno_list, val_img_list, val_anno_list = make_datapath_list(rootpath=rootpath)

print(train_img_list[0])
print(train_anno_list[0])

./data/VOCdevkit/VOC2012/JPEGImages/2007_000032.jpg
./data/VOCdevkit/VOC2012/SegmentationClass/2007_000032.png


## Dataset の作成
./utils/data_augumentation.py から必要なファイルをインポートして DataTransform クラスを作成する．  
はじめに，対象の画像とアノテーションデータをセットで変換する必要があるため，その処理を行う Compose クラスを用意する．  
次に，訓練データに対してデータオーギュメンテーションを行う．
データオーギュメンテーションでは，まず Scale クラスで画像の縮小・拡大する．
Scale クラスでは0.5〜1.5倍の中でランダムに画像サイズを変換し，元の画像サイズより小さくなった場合にはもとの大きさになるように黒色で埋め，大きくなった場合には元の大きさと同じとなるように適当な位置を切り抜いてくる．
次に，RndomRotation クラスで-10から10度の範囲で回転させる．
さらに，RandomMirror で1/2の確率で左右を反転させる．
その後 Resize クラスで指定した画像サイズに変換し，最後に Normalize_Tensor クラスで画像を PyTorch のテンソル型に変換し色情報の標準化を行う．  
ちなみに，VOC2012 のアノテーションデータでは物体の協会にラベル255（’ambigious’）が設定されているが，ここでは背景クラスとして扱う．
また，セマンティックセグメンテーションではアノテーションデータのラベル0が予め背景クラスとなっている．  
検証データでは，Resize クラスと Normalize_Tensor クラスによる処理のみを適用する．

In [4]:
from utils.data_augumentation import Compose, Scale, RandomRotation, RandomMirror, Resize, Normalize_Tensor

# データの処理を行うクラス
class DataTransform():
    """
    画像とアノテーションの前処理クラス。訓練時と検証時で異なる動作をする
    画像のサイズを input_size x input_size にし，訓練時はデータオーギュメンテーションする。

    Attributes
    ----------
    input_size : int
        リサイズ先の画像の大きさ
    color_mean : (R, G, B)
        各色チャネルの平均値
    color_std : (R, G, B)
        各色チャネルの標準偏差
    """
    
    def __init__(self, input_size, color_mean, color_std):
        self.data_transform = {
            "train": Compose({
                Scale(scale=[0.5, 1.5]),                # 画像の拡大・縮小
                RandomRotation(angle=[-10, 10]),        # 回転
                RandomMirror(),                         # 左右の反転
                Resize(input_size),                     # input_size にリサイズ
                Normalize_Tensor(color_mean, color_std) # 色情報の標準化・テンソル化
            }),
            "val": Compose({
                Resize(input_size),
                Normalize_Tensor(color_mean, color_std)
            })
        }
        
    def __call__(self, phase, img, anno_class_img):
        return self.data_transform[phase](img, anno_class_img)
    

# Dataset クラス
class VOCDataset(data.Dataset):
    """
    VOC2012 の Dataset を作成するクラス，PyTorch の Dataset クラスを継承

    Attributes
    ----------
    img_list : リスト
        画像のパスを格納したリスト
    anno_list : リスト
        アノテーションへのパスを格納したリスト
    phase : 'train' or 'test'
        学習か訓練かを設定する。
    transform : object
        前処理クラスのインスタンス
    """
    
    def __init__(self, img_list, anno_list, phase, transform):
        self.img_list = img_list
        self.anno_list = anno_list
        self.phase = phase
        self.transform = transform
        
    
    def __len__(self):
        ''' 画像の枚数を返す '''
        return len(self.img_list)
    
    
    def __getitem__(self, index):
        ''' 前処理した画像の Tensor 形式のデータとアノテーションを取得 '''
        img, anno_class_img = self.pull_item(index)
        return img, anno_class_img
    
    
    def pull_item(self, index):
        ''' 画像の Tensor 形式のデータ、アノテーションを取得する '''
        
        # 1. 画像読み込み
        image_file_path = self.img_list[index]
        img = Image.open(image_file_path)  # [高さ][幅][色 RGB]
        
        # 2. アノテーション画像の読み込み
        anno_file_path = self.anno_list[index]
        anno_class_img = Image.open(anno_file_path) # [高さ][幅]
        
        # 3. 前処理の実施
        img, anno_class_img = self.transform(self.phase, img, anno_class_img)
        
        return img, anno_class_img

In [5]:
# 動作確認
# (RGB) の色の平均値と標準偏差
color_mean = (0.485, 0.456, 0.406)
color_std = (0.229, 0.224, 0.225)

# データセットの作成
train_dataset = VOCDataset(train_img_list, train_anno_list, phase="train",
                          transform=DataTransform(input_size=475, color_mean=color_mean, color_std=color_std))
val_dataset = VOCDataset(val_img_list, val_anno_list, phase="val",
                         transform=DataTransform(input_size=475, color_mean=color_mean, color_std=color_std))

# データの取り出し
print(val_dataset.__getitem__(0)[0].shape)
print(val_dataset.__getitem__(0)[1].shape)
print(val_dataset.__getitem__(0))

torch.Size([3, 475, 475])
torch.Size([475, 475])
(tensor([[[ 1.6667,  1.5125,  1.5639,  ...,  1.7523,  1.6667,  1.7009],
         [ 1.5810,  1.4269,  1.4783,  ...,  1.7009,  1.6153,  1.6495],
         [ 1.5639,  1.4098,  1.4440,  ...,  1.6838,  1.5982,  1.6324],
         ...,
         [-0.4739, -0.4911, -0.5424,  ...,  1.2557,  1.1872,  1.2214],
         [-0.5596, -0.4911, -0.4911,  ...,  1.2385,  1.1872,  1.2214],
         [-0.6281, -0.3883, -0.3369,  ...,  1.2385,  1.1872,  1.2214]],

        [[ 1.8333,  1.6758,  1.7283,  ...,  1.9209,  1.8333,  1.8683],
         [ 1.7458,  1.5882,  1.6408,  ...,  1.8683,  1.7808,  1.8158],
         [ 1.7283,  1.5707,  1.6057,  ...,  1.8508,  1.7633,  1.7983],
         ...,
         [-0.5826, -0.6001, -0.6527,  ...,  1.4132,  1.3431,  1.3431],
         [-0.6702, -0.6001, -0.6001,  ...,  1.3957,  1.3431,  1.3431],
         [-0.7402, -0.4951, -0.4426,  ...,  1.3957,  1.3431,  1.3431]],

        [[ 2.0474,  1.8905,  1.9428,  ...,  2.1346,  2.0474,  2.08

## DataLoader の作成
最後に DataLoader を作成する．
第2章と異なり，アノテーションデータのサイズがデータごとに変化しないため，PyTorch の DataLoader クラスをそのまま使用できる．

In [7]:
batch_size = 8

train_dataloader = data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
dataloaders_dict = {"train": train_dataloader, "val": val_dataloader}

In [8]:
# 動作確認
batch_iterator = iter(dataloaders_dict["val"])  # イテレータに変換
imges, anno_class_imges = next(batch_iterator)  # 最初の要素を取り出す
print(imges.size())
print(anno_class_imges.size())

torch.Size([8, 3, 475, 475])
torch.Size([8, 475, 475])


In [9]:
import subprocess
subprocess.run(['jupyter', 'nbconvert', '--to', 'python', '3-2_Data_Loader.ipynb'])

CompletedProcess(args=['jupyter', 'nbconvert', '--to', 'python', '3-2_Data_Loader.ipynb'], returncode=0)